<a href="https://colab.research.google.com/github/krithikp2100/ai-ml/blob/main/Bounding_Boxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.
# import kagglehub
# mikoajkoek_traffic_road_object_detection_polish_12k_path = kagglehub.dataset_download('mikoajkoek/traffic-road-object-detection-polish-12k')

# print('Data source import complete.')


In [2]:
from transformers import pipeline
import torch
from accelerate.test_utils.testing import get_backend
from PIL import Image
device, _, _ = get_backend()
checkpoint = "Intel/zoedepth-kitti"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [3]:
!pip install ultralytics

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00
Mounted at /content/drive


In [4]:
import os

base_dir = '/content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

assert os.path.exists(base_dir), f"Base directory not found"
assert os.path.exists(train_dir), f"Train directory not found: {train_dir}"
assert os.path.exists(val_dir), f"Validation directory not found: {val_dir}"
assert os.path.exists(test_dir), f"Test directory not found: {test_dir}"

# traffic-road-object-detection-polish-12k/road_detection/road_detection

In [5]:
import os

for root, dirs, files in os.walk(base_dir):
    print(f"Root: {root}")
    print(f"Directories: {dirs}")
    print(f"Files: {len(files)} files found\n")


Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection
Directories: ['test', 'valid', 'train']
Files: 0 files found

Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection/test
Directories: ['images', 'labels']
Files: 0 files found

Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection/test/images
Directories: []
Files: 484 files found

Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection/test/labels
Directories: []
Files: 484 files found

Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection/valid
Directories: ['labels', 'images']
Files: 0 files found

Root: /content/drive/My Drive/Dataset/traffic-road-object-detection-polish-12k/road_detection/road_detection/valid/labels
Directories: []
Files: 935 files found

Root

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Paths to the dataset directories
train_images_path = os.path.join(test_dir, "images")
train_labels_path = os.path.join(test_dir, "labels")

img_width, img_height = 640, 480  # Assumed image size (adjust based on your actual image size)

# Helper function to parse a single YOLO label line
def parse_yolo_label(label_line):
    label_parts = label_line.strip().split()
    class_id = int(label_parts[0])
    center_x = float(label_parts[1])
    center_y = float(label_parts[2])
    width = float(label_parts[3])
    height = float(label_parts[4])
    return class_id, center_x, center_y, width, height

# Convert YOLO format to bounding box coordinates in image pixel values
def yolo_to_image_coords(center_x, center_y, width, height, img_width, img_height):
    box_x1 = int((center_x - width / 2) * img_width)
    box_y1 = int((center_y - height / 2) * img_height)
    box_x2 = int((center_x + width / 2) * img_width)
    box_y2 = int((center_y + height / 2) * img_height)
    return box_x1, box_y1, box_x2, box_y2

# Load images and corresponding labels
def load_data(images_path, labels_path):
    images = []
    boxes = []
    class_ids = []

    for image_file in os.listdir(images_path):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            # Load image
            image_path = os.path.join(images_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (img_width, img_height))  # Adjust size

            # Append image to list
            images.append(image)

            # Load corresponding label
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(labels_path, label_file)

            with open(label_path, 'r') as file:
                label_data = file.readlines()
                image_boxes = []
                image_class_ids = []

                for label_line in label_data:
                    class_id, center_x, center_y, width, height = parse_yolo_label(label_line)
                    box_x1, box_y1, box_x2, box_y2 = yolo_to_image_coords(center_x, center_y, width, height, img_width, img_height)

                    # Append box and class_id to respective lists
                    image_boxes.append([box_x1, box_y1, box_x2, box_y2])
                    image_class_ids.append(class_id)

                boxes.append(image_boxes)
                class_ids.append(image_class_ids)

    return np.array(images), boxes, class_ids

# Load training data
train_images, train_boxes, train_class_ids = load_data(train_images_path, train_labels_path)

# Example: Inspect the first image, boxes, and classes
print("First Image Shape:", train_images[0].shape)
print("First Image Boxes:", train_boxes[0])
print("First Image Class IDs:", train_class_ids[0])

# Visualize an image with bounding boxes drawn
def visualize_image_with_boxes(image, boxes, class_ids):
    for (box, class_id) in zip(boxes, class_ids):
        x1, y1, x2, y2 = box
        # Draw rectangle and class id
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'Class {class_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # cv2.imshow("Image with Bounding Boxes", image)
    cv2_imshow(image)

# Example visualization
visualize_image_with_boxes(train_images[0].copy(), train_boxes[0], train_class_ids[0])


In [ ]:
def visualize_boxes(i):
  visualize_image_with_boxes(train_images[i].copy(), train_boxes[i], train_class_ids[i])

In [ ]:
sign_ids = [1,6,8,10]
sign_boxes = []
for i in range(len(train_class_ids)):
  sign_boxes.append([])
  for j in range(len(train_class_ids[i])):
    if train_class_ids[i][j] in sign_ids:
      sign_boxes[i].append(train_boxes[i][j])
print(sign_boxes)
print(len(sign_boxes))

In [ ]:
def visualize_image_with_sign_boxes(image, boxes, class_ids):
    for (box, class_id) in zip(boxes, class_ids):
      if class_id in sign_ids:
        x1, y1, x2, y2 = box
        # Draw rectangle and class id
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'Class {class_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # cv2.imshow("Image with Bounding Boxes", image)
    cv2_imshow(image)


def visualize_sign_boxes(i):
  visualize_image_with_sign_boxes(train_images[i].copy(), train_boxes[i], train_class_ids[i])

In [ ]:
i = np.random.randint(484)
visualize_boxes(i)
visualize_sign_boxes(i)

In [ ]:
pil_train_images = [Image.fromarray(image) for image in train_images]
predictions = pipe(pil_train_images[0])
predictions["depth"]